In [1]:
import torch
import dgl
import json

import Constants

In [91]:
def create_empty_dense_graph(n_objects, n_relations, n_attributes, device, obj_bbox_dim=4):
    nodes_source, nodes_target = [], []
    for object_i in range(n_objects):
        for object_j in range(n_objects):    
            nodes_source.append(object_i)
            nodes_target.append(object_j)
    if n_objects == 1:
        print(nodes_source, nodes_target)
    g = dgl.graph(
        (torch.tensor(nodes_source), torch.tensor(nodes_target)),
        num_nodes=n_objects
    )
    g.ndata['bbox'] = torch.zeros(n_objects, obj_bbox_dim)  # dimensions: number of nodes x number of features
    g.ndata['attributes'] = torch.zeros(n_objects, n_attributes) # dimensions: number of nodes  x number of attributes

    if n_objects > 1:
        g.edata['relations'] = torch.zeros(n_objects*n_objects, n_relations)
    return g

In [6]:

# test
N_ATTRIBUTES = 10
OBJ_BBOX_DIM = 4
N_RELATIONS = 10
N_OBJECTS = 5

create_empty_dense_graph(N_OBJECTS, N_RELATIONS, N_ATTRIBUTES, OBJ_BBOX_DIM)

Graph(num_nodes=5, num_edges=20,
      ndata_schemes={'bbox': Scheme(shape=(4,), dtype=torch.float32), 'attributes': Scheme(shape=(10,), dtype=torch.float32)}
      edata_schemes={'relations': Scheme(shape=(10,), dtype=torch.float32)})

In [7]:
import Constants
SCENE_GRAPH_PATH = Constants.ROOT_DIR / 'GraphVQA' / 'sceneGraphs'
QUESTIONS_PATH = Constants.ROOT_DIR / 'GraphVQA' / 'questions'
IMAGES_PATH = Constants.ROOT_DIR / 'GraphVQA' / 'images'
OBJECTS_PATH = Constants.ROOT_DIR / 'GraphVQA' / 'meta_info' / 'objects.json'
ATTRIBUTES_PATH = Constants.ROOT_DIR / 'GraphVQA' / 'meta_info' / 'attributes.json'
RELATIONS_PATH = Constants.ROOT_DIR / 'GraphVQA' / 'meta_info' / 'predicates.json'

In [54]:
# with open(ATTRIBUTES_PATH, 'r') as f:
    # attributes_lst = json.load(f)
attributes_lst = np.unique(all_attributes)

attr_name2attr_idx = {obj:idx for idx, obj in enumerate(attributes_lst)}
attr_idx2attr_name = {idx:obj for idx, obj in enumerate(attributes_lst)}
len(attr_name2attr_idx)

603

In [55]:
with open(OBJECTS_PATH, 'r') as f:
    object_lst = json.load(f)
object_lst

obj_name2obj_idx = {obj:idx for idx, obj in enumerate(object_lst)}
obj_idx2obj_name = {idx:obj for idx, obj in enumerate(object_lst)}
len(obj_name2obj_idx)

600

In [56]:
# with open(RELATIONS_PATH, 'r') as f:
    # relations_lst = json.load(f)
relations_lst = np.unique(all_relations)

relation_name2relation_idx = {obj:idx for idx, obj in enumerate(relations_lst)}
relation_idx2relation_name = {idx:obj for idx, obj in enumerate(relations_lst)}
relation_name2relation_idx

{'about to hit': 0,
 'above': 1,
 'adjusting': 2,
 'approaching': 3,
 'around': 4,
 'at': 5,
 'attached to': 6,
 'balancing on': 7,
 'behind': 8,
 'below': 9,
 'beneath': 10,
 'beside': 11,
 'between': 12,
 'bigger than': 13,
 'biting': 14,
 'blowing out': 15,
 'boarding': 16,
 'brushing': 17,
 'by': 18,
 'carrying': 19,
 'catching': 20,
 'chained to': 21,
 'chasing': 22,
 'chewing': 23,
 'cleaning': 24,
 'climbing': 25,
 'close to': 26,
 'coming down': 27,
 'coming from': 28,
 'coming out of': 29,
 'connected to': 30,
 'contain': 31,
 'cooked in': 32,
 'covered by': 33,
 'covered in': 34,
 'covered with': 35,
 'covering': 36,
 'crossing': 37,
 'cutting': 38,
 'decorated with': 39,
 'displayed on': 40,
 'dragging': 41,
 'draped over': 42,
 'drawn on': 43,
 'drinking': 44,
 'drinking from': 45,
 'driving': 46,
 'driving down': 47,
 'driving on': 48,
 'eating': 49,
 'eating from': 50,
 'eating in': 51,
 'enclosing': 52,
 'entering': 53,
 'exiting': 54,
 'facing': 55,
 'falling off': 56,


In [13]:
with open(SCENE_GRAPH_PATH / 'val_sceneGraphs.json') as f:
    scenegraph_dict_all = json.load(f)
len(scenegraph_dict_all)

10696

In [94]:
def covert_scenegraph_dict_to_scenegraph_object(scenegraph_dict):
    scenegraph_dict_objects = scenegraph_dict['objects']
    # print('number of objects:', len(scenegraph_dict_objects))
    
    # create empty scene graph 
    n_objects = len(scenegraph_dict_objects)
    if n_objects == 0:
        return
    g = create_empty_dense_graph(n_objects, N_RELATIONS, N_ATTRIBUTES, OBJ_BBOX_DIM)
    
    # map objects to node indices
    object_unique_id2object_node_idx = {obj_unique_id:obj_node_idx for obj_node_idx, obj_unique_id in enumerate(scenegraph_dict_objects.keys())}
    object_node_idx2object_unique_id = {obj_node_idx:obj_unique_id for obj_node_idx, obj_unique_id in enumerate(scenegraph_dict_objects.keys())}
    object_node_idx2object_node_name = {obj_node_idx:scenegraph_dict_objects[obj_unique_id]['name'] for obj_node_idx, obj_unique_id in enumerate(scenegraph_dict_objects.keys())}
    for obj_node_idx, (obj_unique_id, obj_dict) in enumerate(scenegraph_dict_objects.items()):
    
        # assign bounding box feature (h, w, x, y)
        h = obj_dict['h'] / scenegraph_dict['height']    # normalized height by image height
        w = obj_dict['w'] / scenegraph_dict['width']     # normalized width by image width
        y = obj_dict['y'] / scenegraph_dict['height']    # normalized height by image height
        x = obj_dict['x'] / scenegraph_dict['width']     # normalized width by image width
        g.ndata['bbox'][obj_node_idx, :] = torch.tensor([h, w, x, y])
    
        # assign attributes
        for attr_name in obj_dict['attributes']:
            g.ndata['attributes'][obj_node_idx, attr_name2attr_idx[attr_name]] = 1.
    
        # assign relation
        # g.edata['relations'][g.edge_ids()]
        for related_obj_dict in obj_dict['relations']:
            # get target object node id
            target_obj_unique_id = related_obj_dict['object']
            target_obj_node_idx = object_unique_id2object_node_idx[target_obj_unique_id]
        
            # get edge id of source->target
            if obj_node_idx == target_obj_node_idx:
                continue
            edge_id = g.edge_ids(obj_node_idx, target_obj_node_idx)
        
            # get name and id of relation
            relation_idx = relation_name2relation_idx[related_obj_dict['name']]
            g.edata['relations'][edge_id][relation_idx] = 1.

In [84]:
all_attributes, all_relations = [], []
for scenegraph_key, scenegraph_dict in scenegraph_dict_all.items():
    for _, obj_dict in scenegraph_dict['objects'].items():
        all_attributes += obj_dict['attributes']

    for _, obj_dict in scenegraph_dict['objects'].items():
        for relation_dict in obj_dict['relations']:
            all_relations += [relation_dict['name']]

In [95]:
N_RELATIONS = len(relation_name2relation_idx)
N_ATTRIBUTES = len(attr_name2attr_idx)
for scenegraph_key, scenegraph_dict in scenegraph_dict_all.items():
    g = covert_scenegraph_dict_to_scenegraph_object(scenegraph_dict)

[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]
[0] [0]


KeyboardInterrupt: 